<a href="https://colab.research.google.com/github/mangeshb1/nlp_experiments/blob/main/hf_WandB_End_to_End_v02_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq transformers datasets wandb

In [ ]:
https://raw.githubusercontent.com/Dcing0727/emotion-detection/main/tweet_emotions.csv

In [ ]:
import wandb

wandb.login()

True

In [ ]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [ ]:
import pandas as pd
imp_ds = pd.read_csv('https://raw.githubusercontent.com/harshita1017/Coronavirus_Tweet_classifier/main/Corona_NLP_test.csv')

In [ ]:
imp_ds.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
df_selected = imp_ds[['OriginalTweet','Sentiment']]

In [ ]:
df_selected.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
df_selected.groupby(by=["Sentiment"]).count()

,OriginalTweet
Sentiment,
Extremely Negative,592
Extremely Positive,599
Negative,1041
Neutral,619
Positive,947


In [ ]:
df_selected['Sentiment'] = df_selected['Sentiment'].replace({'Extremely Negative' : 0, 'Extremely Positive' : 1, 'Negative': 2, 'Neutral':3, 'Positive':4})
df_selected.groupby(by=["Sentiment"]).count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,OriginalTweet
Sentiment,
0,592
1,599
2,1041
3,619
4,947


In [ ]:
from datasets import Dataset

dataset_inp = Dataset.from_pandas(df_selected)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
dataset_inp = dataset_inp.train_test_split(test_size = 0.2)

In [ ]:
dataset_inp

DatasetDict({
    train: Dataset({
        features: ['OriginalTweet', 'Sentiment'],
        num_rows: 3038
    })
    test: Dataset({
        features: ['OriginalTweet', 'Sentiment'],
        num_rows: 760
    })
})

In [ ]:
dataset_inp = dataset_inp.rename_column('Sentiment', 'labels')

In [ ]:
dataset_inp

DatasetDict({
    train: Dataset({
        features: ['OriginalTweet', 'labels'],
        num_rows: 3038
    })
    test: Dataset({
        features: ['OriginalTweet', 'labels'],
        num_rows: 760
    })
})

In [ ]:
dataset_inp = dataset_inp.map(lambda x: tokenizer(x['OriginalTweet'], truncation=True), batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dataset_inp['train'][0]

{'OriginalTweet': 'The coronavirus is doing more for this country than any president. Lowering airline prices, gas prices and no school. fuck this\r\r\nCOVID-19 for president 2020',
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [101,
  1996,
  21887,
  23350,
  2003,
  2725,
  2062,
  2005,
  2023,
  2406,
  2084,
  2151,
  2343,
  1012,
  13845,
  8582,
  7597,
  1010,
  3806,
  7597,
  1998,
  2053,
  2082,
  1012,
  6616,
  2023,
  2522,
  17258,
  1011,
  2539,
  2005,
  2343,
  12609,
  102],
 'labels': 0}

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    report_to = 'wandb',                     # enable logging to W&B
    output_dir = 'topic_classification',    # output directory
    overwrite_output_dir = True,
    evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    learning_rate = 5e-5,                   # we can customize learning rate
    #max_steps = 30000,
    logging_steps = 100,                    # we will log every 100 steps
    eval_steps = 5000,                      # we will perform evaluation every 500 steps
    save_steps = 10000,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
    run_name = 'covid_DS_HF_V01'            # name of the W&B run
)

PyTorch: setting up devices


In [ ]:
from datasets import load_metric
import numpy as np

accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    # metrics from the datasets library have a `compute` method
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model = model,                  # model to be trained
    args = args,                    # training args
    train_dataset=dataset_inp['train'],
    eval_dataset=dataset_inp['test'],
    tokenizer=tokenizer,            # for padding batched data
    compute_metrics=compute_metrics # for custom metrics
)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: OriginalTweet. If OriginalTweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 760
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_accuracy': 0.625,
 'eval_loss': 1.5400340557098389,
 'eval_runtime': 4.2064,
 'eval_samples_per_second': 180.678,
 'eval_steps_per_second': 22.585}

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: OriginalTweet. If OriginalTweet are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3038
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1140


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /tmp/tmp08l1zxse
Configuration saved in /tmp/tmp08l1zxse/config.json
Model weights saved in /tmp/tmp08l1zxse/pytorch_model.bin
tokenizer config file saved in /tmp/tmp08l1zxse/tokenizer_config.json
Special tokens file saved in /tmp/tmp08l1zxse/special_tokens_map.json


TrainOutput(global_step=1140, training_loss=0.21083239316940308, metrics={'train_runtime': 185.7038, 'train_samples_per_second': 49.078, 'train_steps_per_second': 6.139, 'total_flos': 198134740157100.0, 'train_loss': 0.21083239316940308, 'epoch': 3.0})

In [ ]:
wandb.finish()

eval/accuracy,▁██
eval/loss,█▁▁
eval/runtime,▁█▁
eval/samples_per_second,█▁█
eval/steps_per_second,█▁█
train/epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█▁▂▂▃▃▄▄▅▅▆▆▇
train/global_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█▁▁▁▂▃▃▃▄▅▅▅▆▇▇
train/learning_rate,██████████████▇▇▆▅▅▄▄▃▂▂▁
train/loss,█▇▆▆▅▄▄▄▃▃▃▃▂▃▂▂▂▂▂▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
## Read our new model back

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('mangesh/huggingface/model-2cevn3ok:v0', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact model-2cevn3ok:v0, 256.36MB. 7 files... Done. 0:0:0


In [ ]:
path = "/content/artifacts/model-2cevn3ok:v0"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)

loading configuration file /content/artifacts/model-2cevn3ok:v0/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/artifacts/model-2cevn3ok:v0",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.19.1",
  "vocab_size": 30522
}

loading weights file /content/artif

In [ ]:
from transformers import pipeline

In [ ]:
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
clf("Find out how you can protect yourself and loved ones from #coronavirus. ?")

[{'label': 'LABEL_1', 'score': 0.9991645812988281}]